In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline

In [2]:
data = pd.read_csv('/home/moranta/Downloads/archive/melb_data.csv')
# datas.dropna(subset=['BuildingArea'], inplace=True)
# datas.dropna(subset=['Rooms'], inplace=True)
# datas.dropna(subset=['Price'], inplace=True)
# datas.dropna(subset=['Bathroom'], inplace=True)
# datas.dropna(subset=['Landsize'], inplace=True)

# Remplacer les valeurs manquantes par la médiane pour certaines colonnes
# datas['Rooms'].fillna(datas['Rooms'].median(), inplace=True)
# datas['Bathroom'].fillna(datas['Bathroom'].median(), inplace=True)
# datas['Landsize'].fillna(datas['Landsize'].median(), inplace=True)
# datas['BuildingArea'].fillna(datas['BuildingArea'].median(), inplace=True)
# datas['Price'].fillna(datas['Price'].median(), inplace=True)
datas = data.copy()
imputer_data = SimpleImputer(strategy='mean')
datas["Price"] = imputer_data.fit_transform(datas[['Price']])
datas["Rooms"] = imputer_data.fit_transform(datas[['Rooms']])
datas["Bedroom2"] = imputer_data.fit_transform(datas[['Bedroom2']])
datas["Bathroom"] = imputer_data.fit_transform(datas[['Bathroom']])
datas["Landsize"] = imputer_data.fit_transform(datas[['Landsize']])
datas["BuildingArea"] = imputer_data.fit_transform(datas[['BuildingArea']])
datas1 = datas
# datas1 = datas1.drop_duplicates()
datas1 = datas1.drop_duplicates(subset=['BuildingArea'])
datas1 = datas1.drop_duplicates(subset=['Price'])
datas1 = datas1.drop_duplicates(subset=['Rooms'])
datas1 = datas1.drop_duplicates(subset=['Bedroom2'])
datas1 = datas1.drop_duplicates(subset=['Bathroom'])
datas1 = datas1.drop_duplicates(subset=['Landsize'])
datas1.dropna()
# datas.shape

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
2,Abbotsford,5 Charles St,3.0,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
47,Airport West,11 Harrington Rd,6.0,h,725000.0,SP,Barry,7/05/2016,13.5,3042.0,...,3.0,2.0,976.0,186.0,1980.0,Moonee Valley,-37.7239,144.8735,Western Metropolitan,3464.0
124,Alphington,83 Fulham Rd,5.0,h,2840000.0,S,Nelson,12/06/2016,6.4,3078.0,...,4.0,4.0,809.0,165.0,1910.0,Darebin,-37.7750,145.0268,Northern Metropolitan,2211.0
379,Ashburton,115 Ashburn Gr,8.0,h,2950000.0,S,hockingstuart,10/09/2016,11.0,3147.0,...,7.0,4.0,1472.0,618.0,2009.0,Boroondara,-37.8729,145.0788,Southern Metropolitan,3052.0


In [3]:
# Sélectionner les fonctionnalités (features) que vous souhaitez utiliser pour la prédiction
features = ['Rooms', 'Bedroom2', 'Bathroom', 'Landsize', 'BuildingArea']


kf = KFold(n_splits=3, shuffle=True, random_state=42)


In [4]:
# Diviser les données en ensembles d'entraînement et de test
X = datas1[features]
y = datas1['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# X_train=X_train.drop_duplicates(subset=['BuildingArea'], keep='first')



# Normaliser les données
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# Créer un pipeline pour gérer l'imputation et la normalisation
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
])
X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

params = {
    "n_estimators":[100, 200, 500],
    "max_depth": [3, 6, 9],
    "learning_rate": [0.001, 0.01, 0.1, 1]
}
# cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=1)
# X_train['BuildingArea'].duplicated().values.any()

In [5]:
# Définir la validation croisée
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# Créer un modèle de régression par gradient boosting
model =GradientBoostingRegressor(learning_rate=0.1, max_depth=3, n_estimators=100, random_state=42)

# grid_search = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, cv=cv)

# Évaluer le modèle avec la validation croisée
scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')

# Calculer la moyenne et l'écart-type des scores de validation croisée
mean_mse = -scores.mean()
std_mse = scores.std()

In [6]:
# Entraîner le modèle sur l'ensemble d'entraînement
# grid_search.fit(X_train, y_train)
# best_params = grid_search.best_params_
model.fit(X_train, y_train)

score_train = model.score(X_train, y_train)
score_test = model.score(X_test, y_test)
score_test

0.9817873480439442

In [7]:
# y_pred = grid_search.predict(X_test)
# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)


In [8]:
# Calculer l'erreur quadratique moyenne (MSE) sur l'ensemble de test
mse = mean_squared_error(y_pred, y_test)
ecart_type = np.sqrt(mse)
# ecart_type = (mse)**0.5

In [9]:
# Afficher les résultats
# print("Meilleurs hyperparamètres:", best_params)
print("Mean Squared Error:", mse)
#Avec cette structure, vous devriez obtenir les meilleurs hyperparamètres et l'erreur quadratique moyenne (MSE) pour votre modèle. Assurez-vous que le fichier CSV est correctement chargé, que les colonnes spécifiées dans features existent dans le fichier de données, et que les données sont correctement nettoyées pour éviter les valeurs manquantes.


Mean Squared Error: 10040748852.448292


In [10]:
# Afficher les résultats
print(f"Mean Squared Error (MSE) en validation croisée : {mean_mse}")
print(f"Écart-type (standard deviation) en validation croisée : {std_mse}")
print(f"Mean Squared Error (MSE) sur l'ensemble de test : {mse}")
print(f"Écart-type (standard deviation) sur l'ensemble de test : {ecart_type}")

Mean Squared Error (MSE) en validation croisée : 3153900291797.301
Écart-type (standard deviation) en validation croisée : 1342692574373.6597
Mean Squared Error (MSE) sur l'ensemble de test : 10040748852.448292
Écart-type (standard deviation) sur l'ensemble de test : 100203.5371254343
